In [27]:
import torch
import torch.nn as nn

input_feat = torch.tensor([[4, 1, 7, 5], [4, 4, 2, 5], [7, 7, 2, 4], [1, 0, 2, 4]], dtype=torch.float32)

#得到一个 4×4 的输入矩阵，shape 是 [4,4]。
#这里相当于是一个单通道的“图像”。

print(input_feat)
print(input_feat.shape)

tensor([[4., 1., 7., 5.],
        [4., 4., 2., 5.],
        [7., 7., 2., 4.],
        [1., 0., 2., 4.]])
torch.Size([4, 4])


In [28]:
conv2d = nn.Conv2d(1, 1, (2, 2), stride=1, padding='same', bias=True)

#输入通道数 = 1
#输出通道数 = 1
#卷积核大小 = 2×2
#步幅 stride = 1
#padding='same' → 输出 shape 与输入相同 (4×4)
#这里 PyTorch 会随机初始化卷积核参数 weight 和偏置 bias。

print(conv2d.weight)
print(conv2d.bias)

Parameter containing:
tensor([[[[-0.1912, -0.4746],
          [-0.3670, -0.4834]]]], requires_grad=True)
Parameter containing:
tensor([0.1266], requires_grad=True)


### 手动指定卷积核

In [29]:
conv2d = nn.Conv2d(1, 1, (2, 2), stride=1, padding='same', bias=False)

#新建了一个卷积层，并且 关闭 bias (bias=False)

# 卷积核要有四个维度(输入通道数，输出通道数，高，宽)
kernels = torch.tensor([[[[1, 0], [2, 1]]]], dtype=torch.float32)
conv2d.weight = nn.Parameter(kernels, requires_grad=False)
print(conv2d.weight)
print(conv2d.bias)

Parameter containing:
tensor([[[[1., 0.],
          [2., 1.]]]])
None


In [30]:
input_feat = torch.tensor(
    [[4, 1, 7, 5],
     [4, 4, 2, 5],
     [7, 7, 2, 4],
     [1, 0, 2, 4]], dtype=torch.float32
).unsqueeze(0).unsqueeze(0)

#在 PyTorch 里，nn.Conv2d 的输入张量 shape 必须是 [batch_size, channels, height, width]：
#batch_size：一次输入多少张图片（或样本）
#channels：每张图片有多少通道（灰度图=1，彩色图=3）
#height, width：图像的高和宽
#unsqueeze(dim) 会在指定位置增加一个维度（大小=1）
#第一次 .unsqueeze(0)：在最前面加一维 → [1, 4, 4]
#第二次 .unsqueeze(0)：再在最前面加一维 → [1, 1, 4, 4]

# torch.Size([1, 1, 4, 4]) ，这符合 PyTorch 卷积层 nn.Conv2d 的输入格式：(batch_size, channels, height, width
#batch_size = 1（只有一张图片）
#channels = 1（灰度图）
#height, width = 4, 4

print(input_feat)
print(input_feat.shape)

tensor([[[[4., 1., 7., 5.],
          [4., 4., 2., 5.],
          [7., 7., 2., 4.],
          [1., 0., 2., 4.]]]])
torch.Size([1, 1, 4, 4])


In [31]:
output = conv2d(input_feat)  #conv2d 卷积操作
print(output)

tensor([[[[16., 11., 16., 15.],
          [25., 20., 10., 13.],
          [ 9.,  9., 10., 12.],
          [ 1.,  0.,  2.,  4.]]]])


## 深度可分离卷积

深度可分离卷积（Depthwise Separable Convolution）由 Depthwise（DW）和 Pointwise（PW）这两部分卷积组合而成的。
深度可分离卷积就是在效果近似相同的情况下，需要的计算量更少



### 传统卷积

在传统卷积里：

输入通常有多个通道（比如彩色图像有 R、G、B 三个通道）。
一个卷积核（filter）会同时作用在所有通道上，然后输出一个特征图。
如果你要输出很多个通道，就需要很多个卷积核，每个卷积核都对所有输入通道做一次完整的卷积。

👉 问题：参数多、计算量大。

### 深度可分离卷积

它把卷积分成两步：

#### 1.逐通道卷积（Depthwise Convolution）
每个输入通道单独用一个卷积核去处理，不再混合通道。
比如 RGB 三个通道，就分别用三个小卷积核处理，得到三个特征图。
👉 这样只做“空间特征提取”，不做通道融合。

#### 2.逐点卷积（Pointwise Convolution，1x1 卷积）
用 1x1 卷积，把上一步得到的特征图按通道再组合起来。
这里负责“通道之间的信息融合”。

### 直观类比

#### 传统卷积就像：
你在做一道菜时，所有的原料（通道）一次性混合着切、炒、调味，步骤一锅端。

#### 深度可分离卷积就像：
先把每种原料（通道）单独切好（Depthwise），
再用调料（1x1 卷积）把它们混合成一道菜。
👉 这样步骤分开，效率更高。

In [32]:
import torch
import torch.nn as nn

# 生成一个三通道的5x5特征图
x = torch.rand((3, 5, 5)).unsqueeze(0)
print(x.shape)
# 输出：
torch.Size([1, 3, 5, 5])
# 请注意DW中，输入特征通道数与输出通道数是一样的
in_channels_dw = x.shape[1]
out_channels_dw = x.shape[1]
# 一般来讲DW卷积的kernel size为3
kernel_size = 3
stride = 1
# DW卷积groups参数与输入通道数一样
dw = nn.Conv2d(in_channels_dw, out_channels_dw, kernel_size, stride, groups=in_channels_dw)

torch.Size([1, 3, 5, 5])


In [33]:
in_channels_pw = out_channels_dw
out_channels_pw = 4
kernel_size_pw = 1
pw = nn.Conv2d(in_channels_pw, out_channels_pw, kernel_size_pw, stride)
out = pw(dw(x))
print(out.shape)

torch.Size([1, 4, 3, 3])


# 空洞卷积

空洞卷积经常用于图像分割任务当中。图像分割任务的目的是要做到 pixel-wise 的输出，也就是说，对于图片中的每一个像素点，模型都要进行预测。
对于一个图像分割模型，通常会采用多层卷积来提取特征的，随着层数的不断加深，感受野也越来越大


普通卷积核是连续取点的，比如 3×3 卷积就是把周围 9 个点都扫一遍。
而 空洞卷积在卷积核里“挖空”，在采样点之间插入空格（dilation rate），让卷积核看得更远。

### 感受野



![](Images/1.png)

In [ ]:
# 空洞卷积
import torch
import torch.nn as nn

# 生成一个三通道的 5x5 特征图，形状 [N, C, H, W] = [1, 3, 5, 5]
x = torch.rand((3, 5, 5)).unsqueeze(0)
print('input shape:', x.shape)  # torch.Size([1, 3, 5, 5])

# -------------------------------
# 1) 普通空洞卷积（Dilated Convolution）
# -------------------------------
in_channels = x.shape[1]
out_channels = 4          # 随便举例：输出通道设为 4
kernel_size = 3
stride = 1
dilation = 2              # 空洞（膨胀）率，>1 即为空洞卷积
# 为了保持输出空间尺寸与输入一致（same padding 的效果），
# 当 kernel=3 时，padding 通常设置为 dilation
padding = dilation

dilated_conv = nn.Conv2d(
    in_channels=in_channels,
    out_channels=out_channels,
    kernel_size=kernel_size,
    stride=stride,
    padding=padding,
    dilation=dilation,     # 关键参数：空洞率
    bias=True
)

y = dilated_conv(x)
print('dilated conv output shape:', y.shape)
# 说明：当 kernel=3, stride=1, padding=dilation 时，输出 H/W 与输入相同
# 这里是 torch.Size([1, 4, 5, 5])

input shape: torch.Size([1, 3, 5, 5])
dilated conv output shape: torch.Size([1, 4, 5, 5])


![](Images/2.gif)

In [ ]:
# -------------------------------
# 2)（可选）空洞版 Depthwise（逐通道）卷积
#    — 把 groups 设为 in_channels，再加 dilation
# -------------------------------
in_channels_dw = x.shape[1]
out_channels_dw = in_channels_dw  # depthwise：输入通道=输出通道
kernel_size_dw = 3
stride_dw = 1
dilation_dw = 2
padding_dw = dilation_dw          # 同理，为了保持尺寸

dw_dilated = nn.Conv2d(
    in_channels=in_channels_dw,
    out_channels=out_channels_dw,
    kernel_size=kernel_size_dw,
    stride=stride_dw,
    padding=padding_dw,
    dilation=dilation_dw,
    groups=in_channels_dw,  # 关键：逐通道
    bias=True
)

y_dw = dw_dilated(x)
print('dilated depthwise output shape:', y_dw.shape)

dilated depthwise output shape: torch.Size([1, 3, 5, 5])


In [ ]:
#要点速记：
#只要在 nn.Conv2d 里把 dilation 设成 >1 就是空洞卷积。
#若想“尺寸不变”，一般用 padding = dilation（在 kernel_size=3、stride=1 的常见设置下）。
#空洞版 depthwise：在上面再加 groups=in_channels。